In [1]:
import os
from crewai import Agent, Task, Crew, Process

# It can be a local model through Ollama / LM Studio or a remote
# model like OpenAI, Mistral, Antrophic or others (https://docs.crewai.com/how-to/LLM-Connections/)

# Define your agents with roles and goals
researcher = Agent(
  role='Fitness Researcher',
  goal='Uncover leading fitness knowledge and advances.',
  backstory="""You work at a well-known fitness research institution.
  Your expertise lies in identifying emerging trends in fitness and providing powerful fitness advice.
  You have a knack for finding hidden information that can help individuals to grow in terms of power and athleticism.""",
  verbose=True,
  allow_delegation=False,
  # You can pass an optional llm attribute specifying what model you wanna use.
  llm= "ollama/llama3.2"
)
writer = Agent(
  role='Sports Advisor',
  goal='Craft compelling content on sports advancements ad advices.',
  backstory="""You are a renowned sports advisor, known for your insightful and engaging advices and articles.
  You transform complex concepts and advices into compelling narratives.""",
  verbose=True,
  allow_delegation=True,
  llm = "ollama/llama3.2"
)

# Create tasks for your agents
task1 = Task(
  description="""Provided the context and query below, answer the query using the context as necessary.
  Ensure the information you provide is factual and accurate.
  
  Context: {Context}

  User query: {user_query}
  """,
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Given the advice produced by Fitness Researcher, verify the advise and add on any missing information the Fitness Researcher may have missed.
  Format your response in an engaging and educational manner while being elaborate. Try not to be too technical as to avoid sounding too much like an AI.""",
  expected_output="Full response of at least 4 paragraphs",
  agent=writer
)

# Instantiate your crew with a sequential process
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=True,
  process = Process.sequential
)


input_query = {
    "user_query" : "What is the best source of protein?",
    "Context" : ""
}

# Get your crew to work!
result = crew.kickoff(inputs = input_query)

print("######################")
print(result)

# Agent: Fitness Researcher
## Task: Provided the context and query below, answer the query using the context as necessary.
  Ensure the information you provide is factual and accurate.
  
  Context: 

  User query: What is the best source of protein?
  


# Agent: Fitness Researcher
## Final Answer: 
Your final answer must be the great and the most complete as possible, it must be outcome described.

**The Best Source of Protein: A Comprehensive Analysis**

Protein is an essential macronutrient that plays a critical role in building and repairing muscles, organs, and tissues in the body. With numerous food sources available, determining the best source of protein can be overwhelming. Here's a detailed analysis of various protein-rich foods to help you make an informed decision:

**Animal-Based Protein Sources:**

• **Lean Meats:** Chicken breast, turkey breast, lean beef, and pork tenderloin are excellent sources of protein, with approximately 30-40 grams of protein per 3-ounce servin

In [7]:
from single_agent import SingleAgent
from single_task import SingleTask
from crewai import Crew

agent1 = SingleAgent("Principal Software Engineer", "Find bugs in program and suggests innovative ideas.", """
You are a principal software engineer from Google. You are highly skilled and knowledgeable in software engineer.
""")


agent2 = SingleAgent("Principal Software Engineer", "Find bugs in program and suggests innovative ideas.", """
You are a principal software engineer from Google. You are highly skilled and knowledgeable in software engineer.
""")

task1 = SingleTask("Solve the problem provided: {user_query}", "Full paragraphs explaining process.", agent1.agent)
task2 = SingleTask("Solve the problem provided: {user_query}. Given answer from Principal Software Engineer, improve upon it and find any possible flaws.", "Full paragraphs explaining process.", agent2.agent)

crew = Crew(agents = [agent1.agent, agent2.agent], tasks = [task1.task, task2.task])

crew.kickoff(inputs = {"user_query" : "How do I train an audio driven face animation for real-time applications?"})

CrewOutput(raw='To train an audio-driven face animation for real-time applications, we can follow a multi-step process that involves data collection, feature extraction, model training, and deployment. First, we need to collect a large dataset of audio files with corresponding face animations, which can be achieved through semi-supervised or unsupervised learning methods.\n\nNext, we extract features from the audio files using techniques such as convolutional neural networks (CNNs) or recurrent neural networks (RNNs), which can provide more accurate representations of the audio data. These features can be used to represent the audio data in a way that is useful for machine learning algorithms.\n\nAfter collecting and preprocessing the data, we can train a machine learning model using a supervised learning approach. A suitable algorithm for this task could be a CNN or RNN that takes the extracted audio features as input and predicts the corresponding face animation parameters.\n\nTo fur

In [ ]:
with open("context.txt", "a") as f:
    query = "What are you doing?"
    result = crew.kickoff(inputs = {"user_query" : query})

    f.write(f"User query: {query}" + "\n" + "Response: " + result.tasks_output)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np

prompt = lambda user_input, context: f"""
You are my personal assistant who specializes in computer science. Using the context given below, answer to my query using the context.

Context: {context}

My query: {user_input}
"""

user_query = "What is your name?"
top_k = 5

model_name = "Qwen/Qwen2-1.5B"
llm = AutoModelForCausalLM.from_pretrained(model_name, device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

sentence_encoder = SentenceTransformer("BAAI/bge-small-en-v1.5")

with open("context.txt", "r+") as f:
    context_doc = f.read()
    context_paragraphs = context_doc.split("\n\n")

    context_embedding = sentence_encoder.encode(context_paragraphs, normalize_embeddings = True)
    input_embedding = sentence_encoder.encode(user_query, normalize_embeddings = True)
    similarities = np.dot(context_embedding, input_embedding.T) # Calculate similarity scores.
    top_k_indices = np.argsort(similarities)[::-1][::top_k] # Sort from greatest to least and grab top_k greatest.

    top_k_context = np.array(context_paragraphs)[top_k_indices] # Grab top_k indices.
    
    # Creates context string.
    context = ""
    for para in top_k_context:
        context += para + "\n\n"

    my_input = prompt(user_query, context)

    tokens = tokenizer(my_input, return_tensors="pt")
    result = tokenizer.decode(llm.generate(input_ids = tokens["input_ids"], max_new_tokens = 200)[0])
    print(result)

    new_subcontext = f"User query: {user_query}\nMy response: {result}"
    f.write(new_subcontext)
    context_paragraphs.append(new_subcontext)

Some parameters are on the meta device because they were offloaded to the disk and cpu.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
import numpy as np

prompt = lambda user_input, context: f"""
You are my personal assistant who specializes in computer science. Using the context given below, answer to my query using the context.

Context: {context}

My query: {user_input}
"""

user_query = "What is your name?"
top_k = 5

model_name = "Qwen/Qwen2-1.5B"
llm = AutoModelForCausalLM.from_pretrained(model_name, device_map = "auto")
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)

sentence_encoder = SentenceTransformer("BAAI/bge-small-en-v1.5")

with open("context.txt", "r+") as f:
    context_doc = f.read()
    context_paragraphs = context_doc.split("\n\n")

    context_embedding = sentence_encoder.encode(context_paragraphs, normalize_embeddings = True)
    input_embedding = sentence_encoder.encode(user_query, normalize_embeddings = True)
    similarities = np.dot(context_embedding, input_embedding.T) # Calculate similarity scores.
    top_k_indices = np.argsort(similarities)[::-1][::top_k] # Sort from greatest to least and grab top_k greatest.

    top_k_context = np.array(context_paragraphs)[top_k_indices] # Grab top_k indices.
    
    # Creates context string.
    context = ""
    for para in top_k_context:
        context += para + "\n\n"

    my_input = prompt(user_query, context)

    messages = [
    {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
    {"role": "user", "content": my_input}
    ]

    text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(llm.device)

    generated_ids = llm.generate(**model_inputs, max_new_tokens = 200)
    # generated_ids = [
    # output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    # ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print(response)

    new_subcontext = f"User query: {user_query}\nMy response: {response}"
    f.write(new_subcontext)
    context_paragraphs.append(new_subcontext)